Reference: https://www.geeksforgeeks.org/deep-learning/emotion-detection-using-convolutional-neural-networks-cnns/

In [4]:
import cv2
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# MODEL

## Define MobileNetV2-based emotion model

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

IMG_SIZE = 96  # we can change this later if needed
NUM_CLASSES = 7  # typical basic emotions; adjust later if needed


def build_emotion_model():
    # Pretrained backbone
    base = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights="imagenet",
    )
    base.trainable = False  # freeze for now; we can fine-tune later

    # Simple classifier head
    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


emotion_model = build_emotion_model()
emotion_model.summary()

2025-11-21 22:06:11.028941: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-11-21 22:06:11.029018: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2025-11-21 22:06:11.029022: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2025-11-21 22:06:11.029053: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-21 22:06:11.029063: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_96             │ (None, 3, 3, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │         8,967 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,266,951 (8.65 MB)

 Trainable params: 8,967 (35.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
import numpy as np

# fake batch of 4 RGB images sized 96x96
dummy_batch = np.random.rand(4, IMG_SIZE, IMG_SIZE, 3).astype("float32")

probs = emotion_model.predict(dummy_batch)
print("Output shape:", probs.shape)
print("First example prob sum:", probs[0].sum())
print("First example probs:", probs[0])

2025-11-21 22:07:18.716282: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Output shape: (4, 7)
First example prob sum: 1.0
First example probs: [0.61368805 0.14342226 0.07003041 0.02621308 0.03122267 0.08127619
 0.03414739]


In [7]:
from pathlib import Path
import tensorflow as tf

IMG_SIZE = 96  # same as model
BATCH_SIZE = 64

DATA_ROOT = Path("data/RAF-DB/DATASET")
train_dir = DATA_ROOT / "train"
test_dir = DATA_ROOT / "test"

AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",  # use folder names 1..7
    label_mode="int",
    image_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    shuffle=True,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="int",
    image_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    shuffle=False,
)

print("Class names:", train_ds.class_names)

# Prefetch for speed
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

Found 12271 files belonging to 7 classes.
Found 3068 files belonging to 7 classes.
Class names: ['1', '2', '3', '4', '5', '6', '7']


In [8]:
images, labels = next(iter(train_ds))
print("Batch images shape:", images.shape)
print("Batch labels shape:", labels.shape)
print("First 10 labels:", labels[:10].numpy())

Batch images shape: (64, 96, 96, 3)
Batch labels shape: (64,)
First 10 labels: [4 3 4 3 5 3 3 3 3 0]


In [10]:
history = emotion_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.5707 - loss: 1.1624 - val_accuracy: 0.5675 - val_loss: 1.1653
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.5808 - loss: 1.1525 - val_accuracy: 0.5727 - val_loss: 1.1860
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.5804 - loss: 1.1467 - val_accuracy: 0.5450 - val_loss: 1.1954
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.5828 - loss: 1.1388 - val_accuracy: 0.5707 - val_loss: 1.1734
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.5861 - loss: 1.1193 - val_accuracy: 0.5759 - val_loss: 1.1560
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.5823 - loss: 1.1220 - val_accuracy: 0.5658 - val_loss: 1.1666
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.5855 - loss: 1.1210 - val_accuracy: 0.5818 - val_loss: 1.1672
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.5869 - loss: 1.1240 - val_accu

In [11]:
emotion_model.save("emotion_mobilenet_rafdb.h5")

# RUN MODEL ON FAIRFACE